# 🛍️ | Cora-For-Zava: 모델 선택

환영합니다! 이 노트북에서는 표준화된 테스트 데이터셋과 Azure AI 평가 SDK를 사용하여 여러 AI 모델을 평가하는 방법을 안내합니다.

## 🛒 Zava 시나리오

**Cora**는 DIY 애호가를 위한 홈 인테리어 용품을 판매하는 가상의 소매업체 **Zava**의 고객 서비스 챗봇입니다. Cora가 최고의 고객 경험을 제공하도록 하려면 적합한 기본 모델을 선택해야 합니다. Azure OpenAI에는 GPT-4o, GPT-4o-mini, GPT-4 등 다양한 모델이 있으므로, 소매업 사용 사례에 가장 적합한 품질, 안전성 및 성능의 균형을 제공하는 모델을 평가해야 합니다.

## 🎯 완성할 결과물

이 노트북을 마치면 다음을 수행할 수 있습니다.
- ✅ 비교를 위해 여러 Azure OpenAI 모델을 구성했습니다.
- ✅ 평가를 위해 표준화된 테스트 데이터 세트를 로드했습니다.
- ✅ 내장 평가 도구를 사용하여 모델 간 평가를 실행했습니다.
- ✅ 성능 지표(품질, 안전성, 지연 시간)를 분석했습니다.
- ✅ 모델 결과를 비교하여 정보에 기반한 모델 선택 결정을 내렸습니다.

## 💡 학습 내용

- 평가를 위해 여러 모델을 구성하는 방법
- 평가를 위해 테스트 데이터 세트를 로드하는 방법
- 내장 평가 도구를 사용하여 평가를 실행하는 방법
- 모델 성능을 분석하고 비교하는 방법
- Azure AI Foundry 모델 리더보드를 사용하는 방법

> **참고**: 이 노트북은 AI 애플리케이션을 배포하기 전에 필수적인 사전 프로덕션 평가를 보여줍니다.

모델을 비교할 준비가 되셨나요? 시작해 봅시다! 🚀

---

## Step 1: 환경 변수 확인

다음 환경 변수들은 이전 설정 단계에서 `.env` 파일에 이미 구성되어 있어야 합니다:

- **AZURE_OPENAI_API_KEY**: Azure OpenAI API 키
  - 제거하고 SystemManagedIdentity 사용 예정
- **AZURE_OPENAI_ENDPOINT**: Azure OpenAI 서비스 엔드포인트
- **AZURE_OPENAI_API_VERSION**: 사용할 API 버전
- **AZURE_SUBSCRIPTION_ID**: Azure 구독 ID
- **AZURE_RESOURCE_GROUP**: Azure 리소스 그룹 이름
- **AZURE_AI_PROJECT_NAME**: Azure AI Foundry 프로젝트 이름

In [12]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
# Use override=True to reload any changes made to .env
load_dotenv(override=True)

# Verify all required Azure service credentials are available
required_vars = [
    "AZURE_OPENAI_ENDPOINT",
    "AZURE_OPENAI_API_VERSION",
    "AZURE_SUBSCRIPTION_ID",
    "AZURE_RESOURCE_GROUP",
    "AZURE_AI_PROJECT_NAME",
    "AZURE_AI_FOUNDRY_NAME"
]

missing_vars = [var for var in required_vars if not os.environ.get(var)]

if missing_vars:
    raise EnvironmentError(
        f"❌ Missing environment variables: {', '.join(missing_vars)}")

print("✅ All environment variables configured!")

## Step 2: 평가할 모델 정의

평가할 모델 배포 이름 배열을 구성합니다. Azure OpenAI 리소스에 배포된 모델을 기반으로 이 목록에서 모델을 추가하거나 제거할 수 있습니다.

> **팁**: 모델을 배포하기 전에 품질, 안전성, 비용 및 성능 측면에서 비교하려면 [Microsoft Foundry 모델 리더보드](https://learn.microsoft.com/azure/ai-foundry/how-to/benchmark-model-in-catalog)를 활용하세요.

In [13]:
# Define the models to evaluate
# Add or remove model deployment names as needed
models_to_evaluate = [
    "gpt-4o-mini",
    "gpt-4o",
    "gpt-4.1"
]

print(f"✅ Configured {len(models_to_evaluate)} models for evaluation")

## 💡 리더보드를 활용한 모델 선택

사용자 지정 평가를 실행하기 전후로 Azure AI Foundry 모델 리더보드를 활용해 가장 적합한 모델을 선택할 수 있습니다:

**리더보드 접근 방법:**
1. [Microsoft Foundry Portal](https://ai.azure.com)로 이동
2. 왼쪽 패널에서 **Model catalog** 선택
3. Model leaderboards 섹션에서 **Browse leaderboards** 클릭

**비교 가능한 항목:**
- **품질 리더보드**: 추론, Q&A, 코딩, 수학 과제별 정확도 순위
- **안전 리더보드**: 유해 콘텐츠에 대한 저항성 순위
- **비용 리더보드**: 비용 효율성 순위
- **성능 리더보드**: 처리량 및 지연 시간 순위
- **트레이드오프 차트**: 품질 vs 비용, 품질 vs 안전성, 품질 vs 처리량
- **시나리오별**: 챗봇, 코드 생성 등 사용 사례에 적합한 모델 찾기

이것은 `models_to_evaluate` 목록을 좁히는 데 도움이 됩니다!

---

## Step 3: 테스트 데이터셋 로드

평가용 테스트 데이터셋(질의와 예상 응답)을 로드합니다. 이 데이터셋은 모든 모델에 대한 테스트 입력으로 사용됩니다.

In [14]:
import json
import pandas as pd

# Load the evaluation dataset
dataset_path = "22-evaluate-models.jsonl"
test_data = []

with open(dataset_path, "r") as f:
    for line in f:
        test_data.append(json.loads(line))

print(f"✅ Loaded {len(test_data)} test examples from {dataset_path}\n")

# Display as DataFrame for easy viewing
df_test_data = pd.DataFrame(test_data)
print("📊 Test Dataset Preview:")
display(df_test_data)

,query,ground_truth,response
0,When was United States found ?,1776,1600
1,What is the capital of France?,Paris,Paris
2,What type of finish does the durable eggshell ...,The durable eggshell finish paint has a subtle...,The durable eggshell finish paint has a subtle...
3,What product fits standard paint trays for qui...,Disposable plastic liners that fit standard pa...,The product that fits standard paint trays for...
4,"Which paint is recommended for kitchens, bathr...",Washable semi-gloss interior paint for kitchen...,The washable semi-gloss interior paint is reco...


## Step 4: Azure AI 프로젝트 구성

평가를 실행하기 위해 Azure AI 프로젝트 연결을 설정합니다.

In [15]:
from azure.ai.evaluation import AzureOpenAIModelConfiguration
from azure.identity import DefaultAzureCredential

# Create Azure AI project configuration
subscription_id = os.environ.get("AZURE_SUBSCRIPTION_ID")
resource_group_name = os.environ.get("AZURE_RESOURCE_GROUP")
project_name = os.environ.get("AZURE_AI_PROJECT_NAME")
azure_ai_foundry_name = os.environ.get("AZURE_AI_FOUNDRY_NAME")

# Dynamically construct the Azure AI Foundry project URL
azure_ai_project_url = f"https://{azure_ai_foundry_name}.services.ai.azure.com/api/projects/{project_name}"

# Initialize and verify credential
try:
    credential = DefaultAzureCredential()
    # Try to get a token to verify authentication
    token = credential.get_token("https://management.azure.com/.default")
    print("✅ Azure credentials verified successfully!")
except Exception as e:
    print("❌ Azure credentials not found or expired!")
    print("Please run 'az login' in the terminal to authenticate with Azure.")
    raise

print(f"✅ Azure AI Project configured")

## Step 5: 모델 구성 생성

평가하려는 각 모델에 대한 모델 구성 객체를 생성합니다.

In [16]:
# Create model configurations for all models
model_configs = {}

for model_name in models_to_evaluate:
    model_configs[model_name] = AzureOpenAIModelConfiguration(
        azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
        azure_deployment=model_name,
        credential=DefaultAzureCredential(),
        api_version=os.environ.get("AZURE_OPENAI_API_VERSION"),
    )

print(f"✅ Created configurations for {len(model_configs)} models")

## Step 6: 타겟 함수 정의

질의를 받아 특정 모델의 응답을 반환하는 함수를 만듭니다. 이는 평가자가 사용할 것입니다.

In [17]:
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider


def create_target_function(model_name):
    """Create a target function for a specific model"""

    def target_function(query: str, ground_truth: str = None, response: str = None) -> dict:
        """Generate response from the model"""
        client = AzureOpenAI(
            azure_ad_token_provider=get_bearer_token_provider(
                DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"),
            api_version=os.environ.get("AZURE_OPENAI_API_VERSION"),
            azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT")
        )

        # Call the model with the query
        response = client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": "You are a helpful assistant that answers questions accurately and concisely."},
                {"role": "user", "content": query}
            ],
            temperature=0.7,
            max_tokens=800
        )

        return {
            "response": response.choices[0].message.content
        }

    return target_function


print("✅ Target function factory created")

## Step 7: Evaluator 구성

모델 성능을 평가하기 위해 사용할 평가자를 설정합니다. 품질과 안전성 지표를 위해 [내장 평가도구](https://learn.microsoft.com/azure/ai-foundry/concepts/observability#what-are-evaluators)를 사용합니다.

**품질 평가자** (AI 지원):
- **Relevance**: 응답이 질문에 얼마나 적절한지 평가 (1-5 스케일)
- **Coherence**: 출력의 흐름과 자연스러움 평가 (1-5 스케일)
- **Fluency**: 언어 능력 및 문법 정확도 평가 (1-5 스케일)

**안전성 평가자** (컨텐츠 안전):
- **Violence**: 응답에서 폭력적 컨텐츠 탐지 (0-7, 값이 낮을수록 안전)
- **Hate/Unfairness**: 증오/불공정성 탐지 (0-7)
- **Self-Harm**: 자해 관련 내용 탐지 (0-7)
- **Sexual**: 성적 컨텐츠 탐지 (0-7)

> **참고**: 저희는 컨텍스트나 정답 데이터가 필요 없는 관련성, 일관성, 유창성 평가 도구를 사용했습니다. 간단한 데이터셋에서 제공하지 않는 추가 컨텍스트가 필요한 근거성 평가 도구는 제거되었습니다.

> [평가 메트릭](https://learn.microsoft.com/azure/machine-learning/prompt-flow/concept-model-monitoring-generative-ai-evaluation-metrics) 및 사용 사례에 대해 자세히 알아보세요.

In [18]:
from azure.ai.evaluation import (
    GroundednessEvaluator,
    RelevanceEvaluator,
    CoherenceEvaluator,
    FluencyEvaluator,
    ViolenceEvaluator,
    HateUnfairnessEvaluator,
    SelfHarmEvaluator,
    SexualEvaluator
)

# Create a judge model configuration for evaluators
judge_model_config = AzureOpenAIModelConfiguration(
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    azure_deployment="gpt-4o",  # Use a capable model as judge
    api_version=os.environ.get("AZURE_OPENAI_API_VERSION"),
)

# Initialize quality evaluators
groundedness_eval = GroundednessEvaluator(model_config=judge_model_config)
relevance_eval = RelevanceEvaluator(model_config=judge_model_config)
coherence_eval = CoherenceEvaluator(model_config=judge_model_config)
fluency_eval = FluencyEvaluator(model_config=judge_model_config)

# Initialize safety evaluators (using azure_ai_project_url instead of dictionary)
violence_eval = ViolenceEvaluator(
    azure_ai_project=azure_ai_project_url, credential=credential)
hate_unfairness_eval = HateUnfairnessEvaluator(
    azure_ai_project=azure_ai_project_url, credential=credential)
self_harm_eval = SelfHarmEvaluator(
    azure_ai_project=azure_ai_project_url, credential=credential)
sexual_eval = SexualEvaluator(
    azure_ai_project=azure_ai_project_url, credential=credential)

print("✅ Evaluators configured:")
print("   Quality: Groundedness, Relevance, Coherence, Fluency")
print("   Safety: Violence, Hate/Unfairness, Self-Harm, Sexual")

In [19]:
# Test with a single model and single prompt
import tempfile
import sys
from datetime import datetime

print("🧪 Running configuration test...", flush=True)
print("=" * 60, flush=True)

# Select first model for testing
test_model = models_to_evaluate[0]
print(f"\n📋 Test Model: {test_model}", flush=True)

# Create a simple test dataset with one example
test_example = {
    "query": "What is the capital of France?",
    "ground_truth": "Paris",
    "response": "Paris"
}

# Save test example to a temporary file
test_file = tempfile.NamedTemporaryFile(
    mode='w', suffix='.jsonl', delete=False)
test_file.write(json.dumps(test_example) + '\n')
test_file.close()

print(f"📝 Test Query: {test_example['query']}", flush=True)

try:
    # Create target function for test model
    test_target_fn = create_target_function(test_model)

    # Test the target function
    print("\n1️⃣ Testing target function...", flush=True)
    sys.stdout.flush()

    test_result = test_target_fn(**test_example)
    print(
        f"   ✅ Target function returned: {test_result['response'][:100]}...", flush=True)

    # Test evaluation with minimal evaluators AND portal publishing
    print("\n2️⃣ Testing evaluation pipeline with portal publishing...", flush=True)
    sys.stdout.flush()

    from azure.ai.evaluation import evaluate

    test_timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    test_eval_result = evaluate(
        data=test_file.name,
        target=test_target_fn,
        evaluators={
            "relevance": relevance_eval,
            "coherence": coherence_eval,
        },
        evaluator_config={
            "default": {
                "query": "${data.query}",
                "response": "${target.response}",
            }
        },
        # Publish to portal for verification (using URL format)
        azure_ai_project=azure_ai_project_url,
        evaluation_name=f"22-evaluate-models-TEST_{test_model}_{test_timestamp}"
    )

    print(f"\n   ✅ Evaluation completed successfully!", flush=True)
    print(f"   📊 Sample metrics:", flush=True)
    print(
        f"      - Relevance: {test_eval_result['metrics'].get('relevance', 'N/A')}", flush=True)
    print(
        f"      - Coherence: {test_eval_result['metrics'].get('coherence', 'N/A')}", flush=True)

    # Show portal URL if available
    if test_eval_result.get('studio_url'):
        print(f"\n   🌐 View test results in portal:", flush=True)
        print(f"      {test_eval_result['studio_url']}", flush=True)

    print("\n" + "=" * 60, flush=True)
    print("✅ Configuration test PASSED! Ready to run full evaluation.", flush=True)
    print("=" * 60, flush=True)
    sys.stdout.flush()

except Exception as e:
    print(f"\n❌ Configuration test FAILED!", flush=True)
    print(f"Error: {str(e)}", flush=True)
    print("\nPlease fix the configuration before proceeding to Step 8.", flush=True)
    print("=" * 60, flush=True)
    sys.stdout.flush()
    raise
finally:
    # Clean up temporary file
    import os
    if os.path.exists(test_file.name):
        os.unlink(test_file.name)
    print("\n🧹 Temporary test file cleaned up.", flush=True)

## Step 8: 평가 실행

이제 `evaluate()` 함수를 사용해 테스트 데이터셋으로 각 모델을 평가합니다. 이는 성능, 품질, 안전성에 대한 종합적인 지표를 생성합니다.

각 평가 작업은:
- 데이터셋의 모든 예시에 대해 모델을 테스트
- 심판 모델을 사용해 품질 지표 계산
- Azure AI Content Safety로 안전성 평가
- 평가 시간 추적(지연 시간 대리 지표로 사용)
- **결과를 Microsoft Foundry 포털에 게시**해 시각화 가능
- **상세 결과를 로컬에 저장**해 오프라인 분석 가능

In [ ]:
from azure.ai.evaluation import evaluate
import time
from datetime import datetime
import os

# Create output directory for evaluation results
output_dir = "./22-evaluate-models-results"
os.makedirs(output_dir, exist_ok=True)

# Store results for each model
evaluation_results = {}

print(f"🚀 Starting evaluation of {len(models_to_evaluate)} models...")
print(f"   Test dataset size: {len(test_data)} examples")
print(f"   Output directory: {output_dir}\n")

for model_name in models_to_evaluate:
    print(f"📊 Evaluating model: {model_name}")
    start_time = time.time()

    try:
        # Create target function for this model
        target_fn = create_target_function(model_name)

        # Create output path for this evaluation
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        output_path = os.path.join(
            output_dir, f"22-evaluate-models_{model_name}_{timestamp}")

        # Run evaluation with both portal publishing and local output
        result = evaluate(
            data=dataset_path,
            target=target_fn,
            evaluators={
                "relevance": relevance_eval,
                "coherence": coherence_eval,
                "fluency": fluency_eval,
                "violence": violence_eval,
                "hate_unfairness": hate_unfairness_eval,
                "self_harm": self_harm_eval,
                "sexual": sexual_eval,
            },
            evaluator_config={
                "default": {
                    "query": "${data.query}",
                    "response": "${target.response}",
                }
            },
            # Publish to Azure AI Foundry portal for visualization (using URL format)
            azure_ai_project=azure_ai_project_url,
            # Save detailed results locally
            output_path=output_path,
            # Optional: provide evaluation name for easier tracking in portal
            evaluation_name=f"22-evaluate-models_{model_name}_{timestamp}"
        )

        elapsed_time = time.time() - start_time

        # Store results with both portal and local file information
        evaluation_results[model_name] = {
            "metrics": result["metrics"],
            "evaluation_time": elapsed_time,
            "studio_url": result.get("studio_url"),
            "output_path": output_path
        }

        print(f"   ✅ Completed in {elapsed_time:.2f} seconds")
        print(f"   📊 Portal URL: {result.get('studio_url', 'N/A')}")
        print(f"   💾 Local results: {output_path}\n")

    except Exception as e:
        print(f"   ❌ Error evaluating {model_name}: {str(e)}\n")
        evaluation_results[model_name] = {
            "error": str(e),
            "evaluation_time": time.time() - start_time
        }

print("✅ All evaluations complete!")
print(f"\n📁 All results saved to: {output_dir}/")
print(f"🌐 View results in Azure AI Foundry portal using the URLs above")

✅ Loaded 5 test examples from 22-evaluate-models.jsonl

📊 Test Dataset Preview:
✅ Azure credentials verified successfully!
✅ Azure AI Project configured
✅ Created configurations for 3 models
✅ Target function factory created
✅ Evaluators configured:
   Quality: Groundedness, Relevance, Coherence, Fluency
   Safety: Violence, Hate/Unfairness, Self-Harm, Sexual
🧪 Running configuration test...

📋 Test Model: gpt-4o-mini
📝 Test Query: What is the capital of France?

1️⃣ Testing target function...
   ✅ Target function returned: The capital of France is Paris....

2️⃣ Testing evaluation pipeline with portal publishing...
2026-01-07 11:09:19 +0000 281471671587280 execution.bulk     INFO     Finished 1 / 1 lines.
2026-01-07 11:09:19 +0000 281471671587280 execution.bulk     INFO     Average execution time for completed lines: 1.47 seconds. Estimated time for incomplete lines: 0.0 seconds.
======= Run Summary =======

Run name: "create_target_function__locals__target_function_20260107_110918_344

Aggregated metrics for evaluator is not a dictionary will not be logged as metrics


2026-01-07 11:09:23 +0000 281471671587280 execution.bulk     INFO     Finished 1 / 1 lines.
2026-01-07 11:09:23 +0000 281471671587280 execution.bulk     INFO     Average execution time for completed lines: 3.47 seconds. Estimated time for incomplete lines: 0.0 seconds.
======= Run Summary =======

Run name: "coherence_20260107_110920_374449"
Run status: "Completed"
Start time: "2026-01-07 11:09:20.374449+00:00"
Duration: "0:00:04.017771"



Aggregated metrics for evaluator is not a dictionary will not be logged as metrics


======= Combined Run Summary (Per Evaluator) =======

{
    "relevance": {
        "status": "Completed",
        "duration": "0:00:03.008642",
        "completed_lines": 1,
        "failed_lines": 0,
        "log_path": null
    },
    "coherence": {
        "status": "Completed",
        "duration": "0:00:04.017771",
        "completed_lines": 1,
        "failed_lines": 0,
        "log_path": null
    }
}



   ✅ Evaluation completed successfully!
   📊 Sample metrics:
      - Relevance: N/A
      - Coherence: N/A

   🌐 View test results in portal:
      https://ai.azure.com/resource/build/evaluation/9f74c298-7787-4df3-8b75-658ba244ae14?wsid=/subscriptions/0ce67698-ac36-4c1c-8188-e8336e25f023/resourceGroups/rg-Ignite-haewons/providers/Microsoft.CognitiveServices/accounts/aoai-2trqdbow7ql7q/projects/proj-2trqdbow7ql7q&tid=16b3c013-d300-468d-ac64-7eda0820b6d3

✅ Configuration test PASSED! Ready to run full evaluation.

🧹 Temporary test file cleaned up.


## Step 9: 결과 분석

모든 모델에 대한 주요 지표 비교 요약을 생성합니다.

In [21]:
import pandas as pd

# Prepare data for comparison
comparison_data = []

for model_name, results in evaluation_results.items():
    if "error" in results:
        print(f"⚠️  {model_name}: Evaluation failed - {results['error']}\n")
        continue

    metrics = results["metrics"]

    # The metrics are stored with keys like "relevance.relevance", "coherence.coherence", etc.
    row = {
        "Model": model_name,
        "Eval Time (s)": results['evaluation_time'],
        "Relevance": metrics.get('relevance.relevance', metrics.get('relevance', 0)),
        "Coherence": metrics.get('coherence.coherence', metrics.get('coherence', 0)),
        "Fluency": metrics.get('fluency.fluency', metrics.get('fluency', 0)),
        "Violence": metrics.get('violence.violence_defect_rate', metrics.get('violence', 0)),
        "Hate/Unfairness": metrics.get('hate_unfairness.hate_unfairness_defect_rate', metrics.get('hate_unfairness', 0)),
        "Self-Harm": metrics.get('self_harm.self_harm_defect_rate', metrics.get('self_harm', 0)),
        "Sexual": metrics.get('sexual.sexual_defect_rate', metrics.get('sexual', 0)),
    }

    comparison_data.append(row)

# Create comparison DataFrame
df_comparison = pd.DataFrame(comparison_data)

print("📊 Model Evaluation Comparison\n")
display(df_comparison)

,Model,Eval Time (s),Relevance,Coherence,Fluency,Violence,Hate/Unfairness,Self-Harm,Sexual
0,gpt-4o-mini,51.421137,4.6,4.2,3.6,0.0,0.0,0.0,0.0
1,gpt-4o,50.138621,4.2,4.0,3.8,0.0,0.0,0.0,0.0
2,gpt-4.1,46.001300,4.8,4.4,3.8,0.0,0.0,0.0,0.0


## Step 10: 성능 요약

평가 결과를 분석해 다양한 지표에서 가장 우수한 성능의 모델을 식별합니다.

In [22]:
from IPython.display import display, Markdown, HTML
import pandas as pd

# Check for successful evaluations
successful_models = {name: results for name,
                     results in evaluation_results.items() if 'error' not in results}
failed_models = {name: results for name,
                 results in evaluation_results.items() if 'error' in results}

if successful_models:
    display(Markdown("## 🏆 Best Performing Models by Metric"))

    # Define evaluator metrics and their optimization direction
    evaluator_metrics = [
        ('relevance.relevance', 'Relevance', True),
        ('coherence.coherence', 'Coherence', True),
        ('fluency.fluency', 'Fluency', True),
        ('violence.violence_defect_rate', 'Violence Safety', False),
        ('hate_unfairness.hate_unfairness_defect_rate',
         'Hate/Unfairness Safety', False),
        ('self_harm.self_harm_defect_rate', 'Self-Harm Safety', False),
        ('sexual.sexual_defect_rate', 'Sexual Safety', False),
    ]

    # Create a dataframe for best models
    best_models_data = []

    for metric_key, display_name, higher_is_better in evaluator_metrics:
        valid_models = {}

        # Collect scores for this metric from all successful models
        for model_name, results in successful_models.items():
            metrics = results['metrics']
            score = metrics.get(metric_key)
            if score is not None:
                valid_models[model_name] = score

        if valid_models:
            # Find best model based on optimization direction
            if higher_is_better:
                best_model_name = max(valid_models, key=valid_models.get)
                best_score = valid_models[best_model_name]
                direction = "↑ Higher is Better"
            else:
                best_model_name = min(valid_models, key=valid_models.get)
                best_score = valid_models[best_model_name]
                direction = "↓ Lower is Better"

            best_models_data.append({
                "Metric": display_name,
                "Best Model": best_model_name,
                "Score": f"{best_score:.3f}",
                "Direction": direction
            })

    df_best = pd.DataFrame(best_models_data)
    display(df_best)

    # Calculate overall best model (based on quality metrics average)
    display(Markdown("---"))
    display(Markdown("## 🌟 Overall Best Model (Quality Metrics Average)"))

    quality_metric_keys = ['relevance.relevance',
                           'coherence.coherence', 'fluency.fluency']
    model_quality_scores = {}

    for model_name, results in successful_models.items():
        metrics = results['metrics']
        scores = []
        for metric_key in quality_metric_keys:
            score = metrics.get(metric_key)
            if score is not None:
                scores.append(score)

        if scores:
            avg_score = sum(scores) / len(scores)
            model_quality_scores[model_name] = {
                'avg_quality': avg_score,
                'eval_time': results['evaluation_time']
            }

    if model_quality_scores:
        best_overall = max(model_quality_scores,
                           key=lambda x: model_quality_scores[x]['avg_quality'])
        best_data = model_quality_scores[best_overall]

        display(Markdown(
            f"🥇 **{best_overall}** - Quality: {best_data['avg_quality']:.3f} | Time: {best_data['eval_time']:.2f}s"))

        # Show all model quality scores for comparison
        display(Markdown(""))

        ranking_data = []
        sorted_models = sorted(model_quality_scores.items(
        ), key=lambda x: x[1]['avg_quality'], reverse=True)

        for rank, (model_name, data) in enumerate(sorted_models, 1):
            medal = "🥇" if rank == 1 else "🥈" if rank == 2 else "🥉" if rank == 3 else ""
            ranking_data.append({
                "Rank": f"{medal} {rank}",
                "Model": model_name,
                "Avg Quality Score": f"{data['avg_quality']:.3f}",
                "Eval Time (s)": f"{data['eval_time']:.2f}"
            })

        df_ranking = pd.DataFrame(ranking_data)
        display(df_ranking)

## 🏆 Best Performing Models by Metric

,Metric,Best Model,Score,Direction
0,Relevance,gpt-4.1,4.800,↑ Higher is Better
1,Coherence,gpt-4.1,4.400,↑ Higher is Better
2,Fluency,gpt-4o,3.800,↑ Higher is Better
3,Violence Safety,gpt-4o-mini,0.000,↓ Lower is Better
4,Hate/Unfairness Safety,gpt-4o-mini,0.000,↓ Lower is Better
5,Self-Harm Safety,gpt-4o-mini,0.000,↓ Lower is Better
6,Sexual Safety,gpt-4o-mini,0.000,↓ Lower is Better


---

## 🌟 Overall Best Model (Quality Metrics Average)

🥇 **gpt-4.1** - Quality: 4.333 | Time: 46.00s

,Rank,Model,Avg Quality Score,Eval Time (s)
0,🥇 1,gpt-4.1,4.333,46.00
1,🥈 2,gpt-4o-mini,4.133,51.42
2,🥉 3,gpt-4o,4.000,50.14


## Step 11: 다음 단계

여러 모델을 성공적으로 평가했습니다! 고려할 다음 단계:

### 📊 결과 조회 위치

- **Azure AI Foundry 포털**: 대화형 시각화(상세 차트 및 비교)
- **포털 URL**: 각 평가에 대해 studio URL 제공, 팀 공유에 유용
- **로컬 파일**: 모든 결과는 `./22-evaluate-models-results/`에 저장됨
- **버전 관리**: JSON 파일을 커밋해 재현성과 시간 경과 추적 여부 확보

### 🏆 모델 리더보드 사용

- **리더보드 탐색**: [품질, 안전성, 비용, 성능별 비교](https://learn.microsoft.com/azure/ai-foundry/how-to/benchmark-model-in-catalog)
- **트레이드오프 분석**: 품질 vs 비용, 품질 vs 안전성 차트 보기
- **시나리오 필터링**: Q&A, 코드 생성, 추론 등 사용 사례에 적합한 모델 찾기
- **포털 접근**: [Microsoft Foundry Model Catalog → Browse Leaderboards](https://aka.ms/model-leaderboards)

---

**수고하셨습니다! 이제 여러 모델에 대한 포괄적 평가 지표를 확보하였습니다.** 🎉
